# Simulador de PLN

In [ ]:
!pip install unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 235 kB 7.5 MB/s 


Você precisa prover o arquivo com os comentários, [daqui](https://www.kaggle.com/luisfredgs/imdb-ptbr), ou utilizar [este notebook](https://www.kaggle.com/tacianotres/kernel68e0f34244), conforme orientado no curso, para que este notebook funcione. O resto é feito automaticamente.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import nltk
from nltk import tokenize
import unidecode
import pickle
from string import punctuation
from sklearn.feature_extraction.text import TfidfVectorizer
import sklearn.externals
import joblib
import json
import codecs

In [ ]:
nltk.download('stopwords')
nltk.download('rslp')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Unzipping stemmers/rslp.zip.


True

In [ ]:
file = '/content/drive/MyDrive/Colab Notebooks/DATA/imdb-reviews-pt-br.csv'
file_modelo = '/content/drive/MyDrive/Colab Notebooks/ML-NLP/modelo_analise_sentimento.pkl'

file_voca = '/content/drive/MyDrive/Colab Notebooks/ML-NLP/modelo_analise_sentimento_vector.pkl'

In [ ]:


# carrega os dados
resenhas = pd.read_csv(file )

# ajustando os sentimentos para 0/1
resenhas['classificacao'] = resenhas['sentiment'].replace(['neg','pos'], [0,1])

# identificando as palavras irrelevantes (stop words)
palavras_irrelevantes = nltk.corpus.stopwords.words('portuguese')

# listando a pontuação
pontuacao = list()
for ponto in punctuation:
    pontuacao.append(ponto)
    
palavras_irrelevantes_e_pontuacao = pontuacao + palavras_irrelevantes

# adicionando casos onde as aspas (simples e duplas) estão no final da frase
palavras_irrelevantes_e_pontuacao.append('".')
palavras_irrelevantes_e_pontuacao.append("'.")
palavras_irrelevantes_e_pontuacao.append('...')

# removendo os acentos
palavras_irrelevantes_e_pontuacao_sem_acento = [unidecode.unidecode(sw) for sw in palavras_irrelevantes_e_pontuacao]

# tokenizador de frases
punct_tokenizer = tokenize.WordPunctTokenizer()

# calculador do stemm das palavras
stemmer = nltk.RSLPStemmer()

In [ ]:
# ajustar as opinioes para o que se espera
# reduzindo palavras para o seu stemm; cuidado: bem lento
frases_processadas = list()
for opiniao in resenhas.text_pt:
    nova_opiniao = list()
    palavras_opiniao = punct_tokenizer.tokenize(unidecode.unidecode(opiniao.lower()))
    for palavra in palavras_opiniao:
        if palavra not in palavras_irrelevantes_e_pontuacao_sem_acento:
            nova_opiniao.append(stemmer.stem(palavra))
    frases_processadas.append(' '.join(nova_opiniao))

resenhas['opiniao_tratada'] = frases_processadas

In [ ]:
# define uma função para criar um classificador de textos, usando TF-IDF e ngrams 1-2
def classificar_texto_tfidf_ngrams(texto, coluna_texto, coluna_classificacao):
    vetorizador = TfidfVectorizer(lowercase=False, ngram_range=(1,2))
    big_bag_of_words = vetorizador.fit_transform(texto[coluna_texto])

    X_train, X_test, Y_train, Y_test = train_test_split(big_bag_of_words,
                                                        texto[coluna_classificacao],
                                                        random_state = 42)

    regressao_logistica = LogisticRegression(solver='lbfgs')
    regressao_logistica.fit(X_train, Y_train)
    
    return regressao_logistica.score(X_test, Y_test), \
        regressao_logistica, \
        vetorizador


In [ ]:
# executa a função para criar os objetos
acuracia, modelo, vetorizador = classificar_texto_tfidf_ngrams(resenhas, 'opiniao_tratada', 'classificacao')

In [ ]:
acuracia #0.8857258390618682

0.8858067124949454

In [ ]:
joblib.dump(modelo, file_modelo)
joblib.dump(vetorizador, file_voca)

Teste

In [ ]:
# simulador
def get_sentindo(modelo, vetorizador, opiniao):
    nova_opiniao = list()
    palavras_opiniao = tokenize.WordPunctTokenizer().tokenize(unidecode.unidecode(opiniao.lower()))
    for palavra in palavras_opiniao:
        if palavra not in palavras_irrelevantes_e_pontuacao_sem_acento:
            nova_opiniao.append(stemmer.stem(palavra))
    opiniao_filtrada = ' '.join(nova_opiniao)

 
    big_bag_of_words = vetorizador.transform([opiniao_filtrada])
 
    return 'Seu comentario é positivo :)' if  (modelo.predict(big_bag_of_words)[0] == 1) else 'Seu comentario é negativo (:'


In [ ]:
def get_modelo_vecto():
    modelo_load  = joblib.load(file_modelo)
    vetor =joblib.load(file_voca)
    return modelo_load , \
          vetor


In [ ]:
modelo , vetor = get_modelo_vecto()



In [ ]:


opiniao = str(input('Sua opnião:'))
print('Sua frase:',opiniao)


Sua opnião:Tu é Brabo
Sua frase: Tu é Brabo


In [ ]:
print( get_sentindo(modelo,vetor , opiniao))

Seu comentario é positivo :)
